# folium excersize


https://nbviewer.jupyter.org/github/python-visualization/folium/tree/master/examples/


In [ ]:
!pip install folium
!pip install geopandas
!pip install geopy

     |████████████████████████████████| 1.0MB 7.0MB/s 
     |████████████████████████████████| 15.3MB 307kB/s 
     |████████████████████████████████| 6.5MB 36.7MB/s 


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import folium
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter


import missingno as msno 
import os

### Importing the data


In [ ]:
# need to mount google drive to access the data
from google.colab import drive
drive.mount('/content/grive')


Mounted at /content/grive


In [ ]:
cd '/content/grive/MyDrive/WDL/Stage_1/data'

/content/grive/MyDrive/WDL/Stage_1/data


In [ ]:
df1 = pd.read_csv('Churn_OD.txt',delimiter="|",encoding = 'Latin1')
df_Individual_journeys = pd.read_csv('Individual_journeys.csv')


In [ ]:
df1.head(10)

,Region_of_Origin,District_of_Origin,County_of_Origin,Region_of_Public_Transportation,District_of_Public_Transportation,County_of_Public_Transportation,Dicofre_ParishCode_of_Public_Transportation,Demand_weight
0,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,LISBOA,110608,0.307323
1,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,LISBOA,110639,0.069997
2,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,LISBOA,110658,0.066059
3,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,LISBOA,110654,0.059847
4,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,LISBOA,110633,0.052341
5,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,LISBOA,110611,0.049032
6,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,AMADORA,111512,0.039779
7,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,LISBOA,110660,0.038691
8,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,LISBOA,110610,0.037827
9,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,LISBOA,110666,0.037800


In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2253 entries, 0 to 2252
Data columns (total 8 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Region_of_Origin                             2253 non-null   object 
 1   District_of_Origin                           2253 non-null   object 
 2   County_of_Origin                             2253 non-null   object 
 3   Region_of_Public_Transportation              2253 non-null   object 
 4   District_of_Public_Transportation            2253 non-null   object 
 5   County_of_Public_Transportation              2253 non-null   object 
 6   Dicofre_ParishCode_of_Public_Transportation  2253 non-null   int64  
 7   Demand_weight                                2253 non-null   float64
dtypes: float64(1), int64(1), object(6)
memory usage: 140.9+ KB


# Mapping exersize
We are going to plot the the county that are listed in the Churn_OD file
1.   Adjust the Churn_OD to have more clean data format
2.   Get the coordinates for the county listed in Churn_OD file
3.   Mapping County on map visualisation 



*1.Adjust the Churn_OD to have more clean data format*

In [ ]:
# formating
df1['District_of_Origin'] = df1['District_of_Origin'].str.lower()
df1['County_of_Origin'] = df1['County_of_Origin'].str.lower()

df1['District_of_Public_Transportation'] = df1['District_of_Public_Transportation'].str.lower()
df1['County_of_Public_Transportation'] = df1['County_of_Public_Transportation'].str.lower()




In [ ]:
# Creating sets of County to compare if all routes have a return route

set1 = set(df1['County_of_Origin'])
set2 = set(df1['County_of_Public_Transportation'])

set3 = set1.difference(set2)
set4 = set2.difference(set1)


We can see that set1 > set2, this mean that not all routes have return "path"

In [ ]:
print('size of set1:',len( set1))
print('size of set2:',len( set2))

size of set1: 34
size of set2: 18


 **set3** show the counties that don't have direct route back to

In [ ]:
for i in set3:
  print (i)

póvoa de varzim
barreiro
palmela
sesimbra
santa maria da feira
cascais
vila franca de xira
vale de cambra
mafra
alcochete
são joão da madeira
setúbal
oliveira de azeméis
montijo
espinho
moita


 **set4** show the counties that don't have direct route back to,
 but as **set4** is empty we assum that all counties in **set2** have return bus route???

In [ ]:
for i in set4:
  print (i)

I want to look for:

*   Number of bus journeys in the same Region and Region to Region 
*   Number of bus journeys from District to District
*   Number of bus journeys from County to County

In [ ]:
# i want to know the exact routes avalable
# df1["Region_Region"] = df1["Region_of_Origin"] + ' to ' + df1["Region_of_Public_Transportation"]
# df1["District_District"] = df1["District_of_Origin"] + ' to ' + df1["District_of_Public_Transportation"]
df1["County_County"] = df1["County_of_Origin"] + ' to ' + df1["County_of_Public_Transportation"]

## 2.Get the coordinates for the county listed in Churn_OD file
 Accuaring County the coordinates 


In [ ]:
## we need the geopandas and geopy to get coordinates of the of the County in the data set 
## Geocoding service is built on top of OpenStreetMap data
!pip install geopandas
!pip install geopy

In [ ]:
#Importing the Nominatim geocoder class 
from geopy.geocoders import Nominatim


## We create 'locator' that holds the Geocoding service, Nominatim will get us the coordinara from  the databas 
locator = Nominatim(user_agent='myGeocoder')
## Then we pass the locator we created to geocode any address, in this example, the Eifel tower address.
location = locator.geocode('Champ de Mars, Paris, France')
print(location)
print('Latitude = {}, Longitude = {}'.format(location.latitude, location.longitude))

Champ de Mars, Rue Saint-Dominique, Quartier du Gros-Caillou, Paris 7e Arrondissement, Paris, Île-de-France, France métropolitaine, 75007, France
Latitude = 48.85614465, Longitude = 2.297820393322227


In [ ]:
## creaeting datafram with proper formant to
df3 = pd.DataFrame()
location_points = pd.DataFrame()
df3['address'] = df1['County_of_Origin'] + ',' + df1['District_of_Origin'] + ',Portugal'

In [ ]:
#for val in df['address'].unique():
#  print (val)
location_points['address'] = df3['address'].unique()
location_points

,address
0,"amadora,lisboa,Portugal"
1,"cascais,lisboa,Portugal"
2,"lisboa,lisboa,Portugal"
3,"loures,lisboa,Portugal"
4,"mafra,lisboa,Portugal"
5,"odivelas,lisboa,Portugal"
6,"oeiras,lisboa,Portugal"
7,"sintra,lisboa,Portugal"
8,"vila franca de xira,lisboa,Portugal"
9,"alcochete,setúbal,Portugal"


In [ ]:
from geopy.extra.rate_limiter import RateLimiter

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
location_points['location'] = location_points['address'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
location_points['point'] = location_points['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [ ]:
# 4 - split point column into latitude, longitude and altitude columns
location_points[['latitude', 'longitude', 'altitude']] = pd.DataFrame(location_points['point'].tolist(), index=location_points.index)

In [ ]:
location_points


,address,location,point,latitude,longitude,altitude
0,"amadora,lisboa,Portugal","(Amadora, Lisboa, Grande Lisboa, Área Metropol...","(38.7595162, -9.223677009460125, 0.0)",38.759516,-9.223677,0.0
1,"cascais,lisboa,Portugal","(Cascais, Lisboa, Grande Lisboa, Área Metropol...","(38.72240025, -9.396909171649877, 0.0)",38.722400,-9.396909,0.0
2,"lisboa,lisboa,Portugal","(Lisboa, Grande Lisboa, Área Metropolitana de ...","(38.7077507, -9.1365919, 0.0)",38.707751,-9.136592,0.0
3,"loures,lisboa,Portugal","(Loures, Lisboa, Grande Lisboa, Área Metropoli...","(38.8578642, -9.175816056948255, 0.0)",38.857864,-9.175816,0.0
4,"mafra,lisboa,Portugal","(Mafra, Lisboa, Grande Lisboa, Área Metropolit...","(38.9369782, -9.3282374, 0.0)",38.936978,-9.328237,0.0
5,"odivelas,lisboa,Portugal","(Odivelas, Lisboa, Grande Lisboa, Área Metropo...","(38.7953835, -9.18782210264363, 0.0)",38.795383,-9.187822,0.0
6,"oeiras,lisboa,Portugal","(Oeiras, Lisboa, Grande Lisboa, Área Metropoli...","(38.7124971, -9.271300382407272, 0.0)",38.712497,-9.271300,0.0
7,"sintra,lisboa,Portugal","(Sintra, Lisboa, Grande Lisboa, Área Metropoli...","(38.8355605, -9.352225245746453, 0.0)",38.835560,-9.352225,0.0
8,"vila franca de xira,lisboa,Portugal","(Vila Franca de Xira, Lisboa, Grande Lisboa, Á...","(38.912368400000005, -8.988824041991643, 0.0)",38.912368,-8.988824,0.0
9,"alcochete,setúbal,Portugal","(Alcochete, Setúbal, Península de Setúbal, Áre...","(38.7560499, -8.9608437, 0.0)",38.756050,-8.960844,0.0


## 3.Mapping County on map visualisation 

In [ ]:
!pip install folium


In [ ]:
import folium

map1 = folium.Map(
    location=[38.7223,9.1393],
    tiles='cartodbpositron',
    zoom_start=12,
)
location_points.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)
map1

In [ ]:
##########

In [ ]:
############

In [ ]:
###########

In [ ]:
#############

In [ ]:
###########

In [ ]:

m=folium.Map(location=[38.7223,9.1393])# #[28.644800, 77.216721]
m

In [ ]:

from branca.element import Figure
fig=Figure(width=550,height=350)

m1=folium.Map(width=550,height=350,location=[28.644800, 77.216721],zoom_start=11,min_zoom=8,max_zoom=14)
fig.add_child(m1)
m1

In [ ]:

m2=folium.Map(width=550,height=350,location=[28.644800, 77.216721],zoom_start=11,min_zoom=8,max_zoom=14)

folium.TileLayer('Stamen Terrain').add_to(m2)
folium.TileLayer('Stamen Toner').add_to(m2)
folium.TileLayer('Stamen Water Color').add_to(m2)
folium.TileLayer('cartodbpositron').add_to(m2)
folium.TileLayer('cartodbdark_matter').add_to(m2)
folium.LayerControl().add_to(m2)
m2

In [ ]:
# Creating Basemap
fig3=Figure(width=550,height=350)
m3=folium.Map(location=[28.644800, 77.216721],tiles='cartodbpositron',zoom_start=11)
fig3.add_child(m3)

#Adding markers to the map
folium.Marker(location=[28.695800, 77.244721],popup='Default popup Marker1',tooltip='Click here to see Popup').add_to(m3)
folium.Marker(location=[28.645800, 77.214721],popup='<strong>Marker3</strong>',tooltip='<strong>Click here to see Popup</strong>').add_to(m3)
folium.Marker(location=[28.655800, 77.274721],popup='<h3 style="color:green;">Marker2</h3>',tooltip='<strong>Click here to see Popup</strong>').add_to(m3)

m3

In [ ]:
# Creating Basemap
fig4=Figure(height=350,width=550)
m4=folium.Map(location=[28.4911091,77.0867361],tiles='cartodbpositron',zoom_start=11)
fig4.add_child(m4)

# Adding Custom Markers
folium.Marker(location=[28.4211091,77.0267361],popup='Custom Marker 1',tooltip='<strong>Click here to see Popup</strong>',icon=folium.Icon(color='red',icon='none')).add_to(m4)
folium.Marker(location=[28.4411091,77.1167361],popup='Custom Marker 2',tooltip='<strong>Click here to see Popup</strong>',icon=folium.Icon(color='green',prefix='glyphicon',icon='off')).add_to(m4)
folium.Marker(location=[28.4511091,77.1667361],popup='Custom Marker 3',tooltip='<strong>Click here to see Popup</strong>',icon=folium.Icon(color='purple',prefix='fa',icon='anchor')).add_to(m4)
# folium.Marker(location=[28.4911091,77.0867361],popup='Custom Marker 4- <b>Analytics Vidhya</b>',tooltip='<strong>Click here to see Popup</strong>',icon=folium.features.CustomIcon('av.png',icon_size=(50,50))).add_to(m4)

m4

In [ ]:
coords_1=[[28.65685,77.21899],[28.63004,77.22135]] 
# [28.65699,77.21898],[28.65699,77.21898],
# [28.65702,77.21875],[28.65702,77.21875],[28.6547,77.21902],[28.65308,77.21921],
# [28.65286,77.21926],[28.65255,77.21938],[28.65227,77.21954],[28.65152,77.22005],
# [28.64655,77.22346],[28.64643,77.22354],[28.64635,77.22357],[28.64623,77.2236],
# [28.64616,77.22363],[28.64612,77.22364],[28.64557,77.22364],[28.64525,77.22365],
# [28.64525,77.22365],[28.64517,77.22366],[28.64515,77.22367],[28.64512,77.22367],
# [28.64504,77.22369],[28.64493,77.22371],[28.64483,77.22373],[28.64462,77.22377],
# [28.6445,77.22381],[28.64444,77.22383],[28.64439,77.22387],[28.64434,77.2239],
# [28.64429,77.22395],[28.64421,77.22403],[28.64417,77.22412],[28.64407,77.22429],
# [28.64391,77.22465],[28.64353,77.22539],[28.64282,77.22678],[28.64282,77.22678],
# [28.64285,77.2268],[28.64287,77.22683],[28.64289,77.22686],[28.64291,77.2269],
# [28.64292,77.22694],[28.64292,77.22697],[28.64292,77.22701],[28.64292,77.22705],
# [28.64292,77.22705],[28.64291,77.22711],[28.64288,77.22716],[28.64285,77.2272],
# [28.64281,77.22724],[28.64276,77.22726],[28.64271,77.22728],[28.64266,77.22728],
# [28.64261,77.22727],[28.64261,77.22727],[28.64257,77.22725],[28.64254,77.22723],
# [28.64251,77.2272],[28.64249,77.22717],[28.64247,77.22713],[28.64245,77.22709],
# [28.64244,77.22704],[28.64244,77.22699],[28.64245,77.22693],[28.64246,77.22689],
# [28.64214,77.22665],[28.64169,77.22632],[28.64163,77.22627],[28.64095,77.22582],
# [28.64009,77.22516],[28.63913,77.22445],[28.63799,77.22352],[28.63794,77.22349],
# [28.63761,77.22327],[28.6375,77.2232],[28.63737,77.22311],[28.63727,77.22304],
# [28.63706,77.22288],[28.63631,77.22233],[28.63528,77.2216],[28.63521,77.22154],
# [28.63521,77.22154],[28.63518,77.22158],[28.63499,77.2218],[28.63471,77.22212],
# [28.63442,77.22238],[28.63411,77.2226],[28.6339,77.22272],[28.63368,77.22282],
# [28.63345,77.2229],[28.63313,77.22297],[28.63281,77.22299],[28.63249,77.22298],
# [28.63217,77.22293],[28.63186,77.22283],[28.63141,77.22256],[28.63125,77.2224],
# [28.63101,77.22227],[28.63082,77.22207],[28.63062,77.22185],[28.63046,77.22162],
# [28.63029,77.22141],[28.63029,77.22141],[28.63026,77.22138],[28.63022,77.22136],
# [28.63018,77.22135],[28.63014,77.22134],[28.6301,77.22134],[28.63006,77.22135],
# [28.63004,77.22135],

coords_2=[[28.63382,77.22045],[28.64022,77.23153]]
# [28.63458,77.221],[28.63514,77.22142],
# [28.63517,77.22146],[28.63519,77.22151],[28.63518,77.22158],[28.63518,77.22158],
# [28.63499,77.2218],[28.63471,77.22212],[28.63442,77.22238],[28.63411,77.2226],
# [28.6339,77.22272],[28.63368,77.22282],[28.63345,77.2229],[28.63313,77.22297],
# [28.63281,77.22299],[28.63249,77.22298],[28.63217,77.22293],[28.63186,77.22283],
# [28.63141,77.22256],[28.63141,77.22256],[28.63129,77.22258],[28.63121,77.22259],
# [28.63113,77.22266],[28.63072,77.22335],[28.63063,77.22353],[28.63059,77.22359],
# [28.63046,77.22381],[28.63043,77.22388],[28.62914,77.22615],[28.62914,77.22615],
# [28.62922,77.22648],[28.62923,77.22651],[28.62924,77.22653],[28.62925,77.22655],
# [28.62926,77.22657],[28.62935,77.22665],[28.62937,77.22667],[28.62938,77.22669],
# [28.62939,77.22672],[28.62943,77.22682],[28.62943,77.22682],[28.62972,77.22701],
# [28.63027,77.22744],[28.63436,77.23048],[28.63456,77.2306],[28.63482,77.23072],
# [28.63511,77.23084],[28.63545,77.23091],[28.63581,77.23095],[28.63656,77.23101],
# [28.63673,77.23102],[28.63824,77.23116],[28.63985,77.23134],[28.64039,77.23144],
# [28.64039,77.23144],[28.64049,77.23147],[28.64038,77.23156],[28.64022,77.23153],

coords_3=[[28.63216,77.21864],[28.64659,77.20495]]
# [28.63225,77.21858],[28.63237,77.21851],
# [28.6325,77.21846],[28.63262,77.21843],[28.63279,77.21841],[28.63295,77.21843],
# [28.63311,77.21846],[28.63326,77.21853],[28.63341,77.21862],[28.63341,77.21862],
# [28.63394,77.21768],[28.6342,77.21722],[28.63425,77.21713],[28.6343,77.21704],
# [28.6343,77.21704],[28.63432,77.21686],[28.63495,77.21581],[28.6357,77.21452],
# [28.63745,77.21151],[28.63951,77.20794],[28.63956,77.20787],[28.63971,77.20763],
# [28.64055,77.2061],[28.64096,77.20533],[28.64106,77.20519],[28.64121,77.20486],
# [28.64132,77.20468],[28.64179,77.20418],[28.64222,77.20381],[28.64254,77.20346],
# [28.64267,77.20333],[28.64277,77.2032],[28.64277,77.2032],[28.64281,77.20314],
# [28.64285,77.20309],[28.64291,77.20306],[28.64297,77.20304],[28.64303,77.20303],
# [28.64309,77.20304],[28.64315,77.20306],[28.6432,77.2031],[28.64324,77.20314],
# [28.64327,77.2032],[28.64327,77.2032],[28.64341,77.20336],[28.64375,77.20351],
# [28.64406,77.20367],[28.64495,77.20426],[28.6459,77.20495],[28.64612,77.2051],
# [28.64612,77.2051],[28.64629,77.2051],[28.64639,77.20508],[28.64648,77.20506],
# [28.64657,77.20497],[28.64659,77.20495],

fig5=Figure(height=550,width=750)
m5=folium.Map(location=[28.644800, 77.216721],tiles='cartodbpositron',zoom_start=14)
fig5.add_child(m5)

In [ ]:
# Creating feature groups
name = 'f1'
FeatureGroupName = "Vehicle 1"
name=folium.FeatureGroup(FeatureGroupName)
f2=folium.FeatureGroup("Vehicle 2")
f3=folium.FeatureGroup("Vehicle 3")

# Adding lines to the different feature groups
line_1=folium.vector_layers.PolyLine(coords_1,popup='<b>Path of Vehicle_1</b>',tooltip=FeatureGroupName,color='blue',weight=10).add_to(name)
line_2=folium.vector_layers.PolyLine(coords_2,popup='<b>Path of Vehicle_2</b>',tooltip='Vehicle_2',color='red',weight=10).add_to(f2)
line_3=folium.vector_layers.PolyLine(coords_3,popup='<b>Path of Vehicle_3</b>',tooltip='Vehicle_3',color='green',weight=10).add_to(f3)

name.add_to(m5)
f2.add_to(m5)
f3.add_to(m5)

In [ ]:

folium.LayerControl().add_to(m5)
m5